In [5]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import ConfigParser
from nltk import ngrams
import hashlib
import numpy.matlib
from sklearn import preprocessing


# $Step 1$Read parameters from the config file

In [6]:
g65_configFileLocation = './config'
Config = ConfigParser.ConfigParser()
Config.read(g65_configFileLocation)
def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1

In [7]:
# set pathes
msd_subset_path = ConfigSectionMap("MainSection")['dataset_location']
msd_code_path = ConfigSectionMap("MainSection")['code_location']
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path'
assert os.path.isdir(msd_code_path),'wrong path'
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

# g65_selctedFeatures stores all the features
g65_selectedFeatures =ConfigSectionMap("MainSection")['features'].split("<g65SEP>")
num_feature = len(g65_selectedFeatures)

# g65_n is number of songs to process
g65_n = int(ConfigSectionMap("MainSection")['number_of_songs_to_process'])

# g65_r is number of rows in each band
g65_r = int(ConfigSectionMap("MainSection")['number_of_rows_each_band'])

# g65_b is number of bands
g65_b = int(ConfigSectionMap("MainSection")['number_of_bands'])

# g65_epsilon is the tolerant difference
g65_epsilon = int(ConfigSectionMap("MainSection")['tolerance_diff'])

#signiture is the raw value of each song
signiture = np.zeros([num_feature,g65_n])

#the hashing algorithm used for hashing bands pieces
g65_hashalgorithm = ConfigSectionMap("MainSection")['hashalgorithm']

In [10]:
import hdf5_getters as GETTERS

In [12]:
# extract all query features into the signiture matrix
s_counter = 0
songsIds = [0]*g65_n
for root, dirs, files in os.walk(msd_subset_data_path):
    files = glob.glob(os.path.join(root,'*'+'.h5'))
    for f in files :
        if s_counter == g65_n:
            break
        else:
            h5 = GETTERS.open_h5_file_read(f)
            f_counter = 0
            songsIds[s_counter] = GETTERS.get_song_id(h5)
            for features in g65_selectedFeatures:
                temp = str(getattr(GETTERS, 'get_'+features)(h5))
                signiture[f_counter,s_counter] = temp
                f_counter += 1
            s_counter += 1
            h5.close()


AttributeError: 'module' object has no attribute 'get_duration,end_of_fade_in,key,loudness,mode,start_of_fade_out,tempo,time_signature,key_confidence'

In [ ]:
signiture = preprocessing.scale(signiture)

In [ ]:
#print signiture

# $Step2$Construct a new signiture matrix M for LSH with cosin distance

In [ ]:
vec = np.random.randn(g65_b*g65_r,num_feature)
vec2 = vec*vec
vecNorm = np.sqrt(vec2.sum(axis=1))
vecNorm = np.matlib.repmat(vecNorm,num_feature,1)
hashV = vecNorm.T/vec
M = np.sign(hashV.dot(signiture))
#print M, M.shape




# $Step3$ Hashing bands to buckets

In [ ]:
BandsBuckets=[]
for i in range(0,g65_b):# for each band
    BandsBuckets.append({})
    for s in range(0,g65_n):
        bandCol = ''.join(str(v) for v in M[g65_r*i:(g65_r*(i+1)-1),s])
        bucketKey = (getattr(hashlib, g65_hashalgorithm)(bandCol)).hexdigest()
        BandsBuckets[i].setdefault(bucketKey,[]).append(s)
        


        

In [ ]:
BandsBuckets
numberOfCandidates=0
for band in BandsBuckets:
    for key, candidatesList in band.iteritems():#looping over lists in band buckets
            if len(candidatesList)>1:
                numberOfCandidates+=len(candidatesList)
print "number of candidates: {}".format( numberOfCandidates )
    
    


# $Step4$ Reporting Duplicates

In [ ]:
duplicates = set() # will eventually hold duplicate pairs of songs
def calculateCosineAngle(v1,v2):
    return  np.dot(v1,v2)/numpy.linalg.norm(v1)/numpy.linalg.norm(v2)

In [ ]:
for i in range(0,g65_b):#looping over bands
    for band in BandsBuckets:
        for key, candidatesList in band.iteritems():#looping over lists in band buckets
            if len(candidatesList)>1:#if a list has more than one item
                for j in range(0,len(candidatesList)-1):#for each two items in the list 
                    for k in range(j+1,len(candidatesList)):
                        cosine = calculateCosineAngle(signiture[:,candidatesList[j]],signiture[:,candidatesList[k]])#we compute the cosine 
                        if np.fabs(cosine)> np.cos( g65_epsilon*np.pi  /180.):# if the cosine is >epsilon
                            duplicates.add((songsIds[j],songsIds[k])) # we report they are a duplicate
                        



# $Step5$ Reporting Duplicates

In [ ]:

print "number of Duplicates: {}".format( len(duplicates)*2)